# Meta programming in ASP

This notebook contains some notes and exercises about Meta programming in ASP.

It is based on section 3 of the paper [1]. 

Before you start, we recommend you to read that section, up to part 3.3.2 (included).

[1] [Kaminski, R., Romero, J., Schaub, T., & Wanko, P. (2023). How to Build Your Own ASP-based System?! Theory Pract. Log. Program., 23(1), 299–361.](https://arxiv.org/pdf/2008.06692.pdf)

## Preliminaries: the aspif format

Consider the following logic program:

In [1]:
%%file example0.lp
{a}.
 b :- a.

Overwriting example0.lp


You can compute its 2 answer sets with the following command.

Option `-V0` reduces the output to **3** lines: one for answer set `{}`, another for answer set `{a,b}`, 
and the last one for the `SATISFIABLE` message.

In [2]:
! clingo example0.lp 0 -V0


a b
SATISFIABLE


You can ground the program using option `--mode=gringo`, that generates a ground program in **aspif** format:

In [3]:
! clingo example0.lp --mode=gringo

asp 1 0 0
1 1 1 1 0 0
1 0 1 2 0 1 1
4 1 a 1 1
4 1 b 1 2
0


The first line is the header.
The last one, with a single `0`, marks the end of the file. 
We explain below in detail the meaning of the other lines.

The script `aspif-pretty-printer.py` gives us a more readable version of the aspif format.

It dismisses the first and the last line, and prints every inner line as a kind of ASP rule or directive
(using numbers instead of symbolic atoms).

In [4]:
! python aspif/aspif-pretty-printer.py example0.lp

{ 1 }.
2 :- 1.
#show a : 1.
#show b : 2.


The script also provides option `--text` that uses the `#show` statements to replace the numbers (`1`, `2`, ...) by symbolic atoms when possible:

In [5]:
! python aspif/aspif-pretty-printer.py --text example0.lp

{ a }.
b :- a.


You can find the specification of the aspif language in Appendix B of [1].

This is the meaning of each line of the previous aspif file:
* `asp 1 0 0` says that the aspif file uses version 1.0.0 of the aspif language.
* `1 1 1 1 0 0` represents the choice rule `{ 1 }.`:
  - The `1` in the 1st position says that the line represents a rule.
  - The `1` in the 2nd position says that the head is a choice.
  - The `1` in the 3rd position says that the head has `1` element.
  - The `1` in the 4th position says that the first element of the head is the atom `1`.
  - The `0` in the 5th position says that the body is a normal body (in contraposition to a weight body).
  - The `0` in the 6th position says that the normal body has `0` elements.
* `1 0 1 2 0 1 1` represents the normal rule `2 :- 1.`:
  - The `1` in the 1st position says that the line represents a rule.
  - The `0` in the 2nd position says that the head is not a choice.
  - The `1` in the 3rd position says that the head has `1` element.
  - The `2` in the 4th position says that the first element of the head is the atom `2`.
  - The `0` in the 5th position says that the body is a normal body (in contraposition to a weight body).
  - The `1` in the 6th position says that the normal body has `1` element.
  - The `1` in the 7th position says that the first element of the body is the atom `2`.
* `4 1 a 1 1` represents the show directive `#show a : 1.`:
  - The `4` in the 1st position says that the line represents a show directive.
  - The `1` in the 2nd position says that the length of the following string is `1` bytes
  - The `a` in the 3rd position is the string to be shown.
  - The `1` in the 4rd position says that the condition has `1` element.
  - The `1` in the 5th position says that the first element of the condition is the atom `1`.
* `4 1 b 1 2` represents similarly the show directive `#show b : 2.`.
* `0` closes the aspif file.


There is another example in the file `aspif/example.lp`, where
every line is followed by its ground instantiation, generated using option `--text` of the script.
We comment it below.

In [6]:
! cat aspif/example.lp

  dom(1..2).
% 1. and #show dom(1).
% 2. and #show dom(2).

  a(X) :- dom(X). 
% 20. and #show a(1).
% 21. and #show a(2).


b(X) :- not a(X), dom(X).
% 

  { c(X) } :- dom(X).
% { c(1) }.
% { c(2) }.

  d(X) :- not c(X), dom(X).
% d(1) :- not c(1).
% d(2) :- not c(2).

  e(X); f(X); g(X) :- c(X), d(X), dom(X).
% e(1); f(1); g(1) :- d(1), c(1).
% e(2); f(2); g(2) :- d(2), c(2).

  { e(X); f(X); g(X) } :- c(X), d(X), dom(X).
% { e(1); f(1); g(1) } :- d(1), c(1).
% { e(2); f(2); g(2) } :- d(2), c(2).

h :- 1 { e(X); f(X) }.
% 18 :- 1 { 1:e(1); 1:e(2); 1:f(1); 1:f(2) }.
% h :- 18.

i :- 1 { e(X); f(X) } 1.
% 14 :- 1 { 1:e(1); 1:e(2); 1:f(1); 1:f(2) }.
% 15 :- 2 { 1:e(1); 1:e(2); 1:f(1); 1:f(2) }.
% 16 :- 14, -15.
% i :- 16.

:- h, i.
% :- i, h.

#show (e(X),f(X),g(X)) : e(X), f(X), g(X).
% 22 :- e(1), f(1), g(1).
% 23 :- e(2), f(2), g(2).
% #show (e(1),f(1),g(1)) : 22.
% #show (e(2),f(2),g(2)) : 23.

  #minimize{ 1@3,X : e(X); 2@3,X : f(X); 3,X : g(X)}.
% #minimize{ 3@0:g(1), 3@0:g(2) }.


You can print just the readable ground instantiation running this command:

In [ ]:
! python aspif/aspif-pretty-printer.py --text aspif/example.lp

Observe that for the **fact** `dom(1)` we have two lines in the aspif file:
* One for the rule without body `1 0 1 1 0 0` that is printed by the script as `1.`
* Another for the show statement `4 6 dom(1) 0` that is printed by the script as `#show dom(1)`.

Using option `--preserve-facts=symtab` clingo would generate the show statement `4 6 dom(1) 1 1` instead of that one,
and the script would just print the fact `dom(1).`

The input language of clingo is very rich, but the grounder manages to translate all rules to a very simple format.

There are only two types of rules: **normal** rules and **weight** rules. 

**Normal** rules have a normal body, 
and have in the head a set of atoms, that can be interpreted as a choice or as a disjunction.
* A disjunction with a single atom is a normal head. 
For example, the ground instance `d(1) :- not c(1).` 
has a normal head `d(1)` that can be seen as a disjunction over the elements of the set `{d(1)}`.
* An empty disjunction is an empty head, in which case the normal rule is an integrity constraint, for example, `:- i, h.`.

**Weight** rules have a weight body, where the weights are positive integers; and have a unique atom as their head.

See how the rule
* ```i :- 1 { e(X); f(X) } 1.```

is not grounded directly to 
* ```i :- 1 { 1:e(1); 1:e(2); 1:f(1); 1:f(2) } 1.```

but is grounded to
* `14 :- 1 { 1:e(1); 1:e(2); 1:f(1); 1:f(2) }.`
* `15 :- 2 { 1:e(1); 1:e(2); 1:f(1); 1:f(2) }.`
* `16 :- 14, -15.`
* `i :- 16.`

to fit into those types of rules. 

The last two rules could have been translated directly into `i :- 14, -15.`. 
The preprocessor of clasp (the solver inside clingo) does that kind of simplifications.
We can run that preprocessor after grounding with the command `clingo --mode=gringo aspif/example.lp --pre`.

We do not dwelve here into the clingo directives `#show`, `#minimize`, etc.

## Reifying a simple example


Let us start with a simple program:

In [7]:
%%file example1.lp
{a}.
#show.

Overwriting example1.lp


We ground it:

In [8]:
! clingo --mode=gringo example1.lp

asp 1 0 0
1 1 1 1 0 0
0


We pretty print the ground instantation:

In [9]:
! python aspif/aspif-pretty-printer.py example1.lp

{ 1 }.


And now we reify it:

In [10]:
! clingo example1.lp --output=reify

atom_tuple(0).
atom_tuple(0,1).
literal_tuple(0).
rule(choice(0),normal(0)).


These facts represent the line `1 1 1 1 0 0` of the ground instantiation (or `{ 1 }.`), by saying that:
* there is a choice rule whose head is the atom tuple `0` and whose body is the literal tuple `0`,
* the atom tuple `0` has the element `1`, and
* the literal tuple `0` has no elements.

If we eliminate the `#show.` directive then the atom `a` will be shown:

In [11]:
%%file example2.lp
{ a }.

Overwriting example2.lp


In [12]:
! clingo --mode=gringo example2.lp

asp 1 0 0
1 1 1 1 0 0
4 1 a 1 1
0


In [13]:
! python aspif/aspif-pretty-printer.py example2.lp

{ 1 }.
#show a : 1.


In [14]:
! clingo example2.lp --output=reify

atom_tuple(0).
atom_tuple(0,1).
literal_tuple(0).
rule(choice(0),normal(0)).
literal_tuple(1).
literal_tuple(1,1).
output(a,1).


The last three facts are new, they represent the line `4 1 a 1 1` (or `#show a : 1`) and say that:
* there is an output statement whose head is the string `a` and whose condition is the literal tuple `1`, and
* the literal tuple `1` consists of the element `1`.

## The meta encoding

In the paper we have seen the meta encoding `meta.lp`:

In [15]:
%%file meta.lp

conjunction(B) :- literal_tuple(B),
        hold(L) : literal_tuple(B, L), L > 0;
    not hold(L) : literal_tuple(B,-L), L > 0.

body(normal(B)) :- rule(_,normal(B)), conjunction(B).
body(sum(B,G))  :- rule(_,sum(B,G)),
    #sum { W,L :     hold(L), weighted_literal_tuple(B, L,W), L > 0 ;
           W,L : not hold(L), weighted_literal_tuple(B,-L,W), L > 0 } >= G.

  hold(A) : atom_tuple(H,A)   :- rule(disjunction(H),B), body(B).
{ hold(A) : atom_tuple(H,A) } :- rule(     choice(H),B), body(B).

#show.
#show T : output(T,B), conjunction(B).

Overwriting meta.lp


Here, we start with a simpler version of it.

In [16]:
%%file meta-simple.lp

conjunction(B) :- literal_tuple(B),
        hold(L) : literal_tuple(B, L), L > 0;
    not hold(L) : literal_tuple(B,-L), L > 0.

  hold(A) : atom_tuple(H,A)   :- rule(disjunction(H),normal(B)), conjunction(B).
{ hold(A) : atom_tuple(H,A) } :- rule(     choice(H),normal(B)), conjunction(B).

#show.
#show T : output(T,B), not literal_tuple(B,_).
#show T : output(T,B),     literal_tuple(B,L), hold(L).

Overwriting meta-simple.lp


This simpler version does not consider weight rules. 
Given this, the rules for `hold(A)` refer to whether a `normal` body `B` holds using directly the predicate `conjunction`.

This simpler version represents differently the `#show` directives.
It considers separately the two cases that may happen with programs generated by clingo using option `--output=reify`:
* either we have `output(T,B)` and `B` has no literals,
* or we have `output(T,B)` and `B` has a unique positive literal.

We use this representation because it is easier to modify in the exercises.

We can try `meta-simple.lp` with our two previous examples. 

Option `-Wnone` eliminates some warnings generated by clingo.

As expected, we obtain two answers for each example. The answers of the first example are empty because nothing is shown in that program.

In [17]:
! clingo example1.lp --output=reify | clingo - meta-simple.lp 0 -Wnone -V0



SATISFIABLE


In [18]:
! clingo example2.lp --output=reify | clingo - meta-simple.lp 0 -Wnone -V0


a
SATISFIABLE


### Exercise 1 
In the next cell, write a modification of `meta-simple.lp` that interprets the choice rules as normal rules.


In [19]:
%%file meta-simple-ex1.lp

conjunction(B) :- literal_tuple(B),
        hold(L) : literal_tuple(B, L), L > 0;
    not hold(L) : literal_tuple(B,-L), L > 0.

  hold(A) : atom_tuple(H,A)   :- rule(disjunction(H),normal(B)), conjunction(B).
{ hold(A) : atom_tuple(H,A) } :- rule(     choice(H),normal(B)), conjunction(B).

#show.
#show T : output(T,B), not literal_tuple(B,_).
#show T : output(T,B),     literal_tuple(B,L), hold(L).

Overwriting meta-simple-ex1.lp


Once you have modified the program, the next cell should return a unique answer set that contains the atom `a`.

In [20]:
! clingo example2.lp --output=reify | clingo - meta-simple-ex1.lp 0 -Wnone -V0


a
SATISFIABLE


Now, modify the `#show` statements of `meta-simple-ex1.lp` in such a way that atoms `T` are shown as `true(T)`.

Once this is done, running the previous cell you should obtain a unique answer set with `true(a)`.

## Extending the example

Consider now the following logic program:

In [21]:
%%file example3.lp
{a}.
 b :- a.

Overwriting example3.lp


In [22]:
! clingo example3.lp --mode=gringo

asp 1 0 0
1 1 1 1 0 0
1 0 1 2 0 1 1
4 1 a 1 1
4 1 b 1 2
0


In [23]:
! python aspif/aspif-pretty-printer.py example3.lp

{ 1 }.
2 :- 1.
#show a : 1.
#show b : 2.


In [24]:
! clingo example3.lp --output=reify

atom_tuple(0).
atom_tuple(0,1).
literal_tuple(0).
rule(choice(0),normal(0)).
atom_tuple(1).
atom_tuple(1,2).
literal_tuple(1).
literal_tuple(1,1).
rule(disjunction(1),normal(1)).
output(a,1).
literal_tuple(2).
literal_tuple(2,2).
output(b,2).


As before, the first four facts represent the choice rule `1 1 1 1 0 0` (or `{ 1 }.`).

The next ones
```
atom_tuple(1).
atom_tuple(1,2).
literal_tuple(1).
literal_tuple(1,1).
rule(disjunction(1),normal(1)).
```
represent the normal rule `1 0 1 2 0 1 1` (or `2 :- 1.`).

The fact
```
output(a,1).
```
together with the previous facts 
```
literal_tuple(1).
literal_tuple(1,1).
```
represents the directive `4 1 a 1 1` (or `#show a : 1.`).

And 
```
literal_tuple(2).
literal_tuple(2,2).
output(b,2).
```
represents the directive `4 1 b 1 2` (or `#show b : 2.`).

We compute the answer sets of `example3.lp`: `{}` and `{a,b}`.

In [25]:
! clingo example3.lp --output=reify | clingo - meta-simple.lp 0 -V0


a b
SATISFIABLE


### Exercise 2
In the next cell, write a modification of `meta-simple.lp` that:
* interprets choice rules as normal rules, and normal rules as choice rules, and
* interprets positive literals in the body as negative literals, and negative literals in the body as positive literals.

Under this interpretation, `example3.lp` becomes:
```
a.
{b} :- not a.
```
and leads to the unique answer set `{a}`.

In [26]:
%%file meta-simple-ex2.lp

conjunction(B) :- literal_tuple(B),
        hold(L) : literal_tuple(B, L), L > 0;
    not hold(L) : literal_tuple(B,-L), L > 0.

  hold(A) : atom_tuple(H,A)   :- rule(disjunction(H),normal(B)), conjunction(B).
{ hold(A) : atom_tuple(H,A) } :- rule(     choice(H),normal(B)), conjunction(B).

#show.
#show T : output(T,B), not literal_tuple(B,_).
#show T : output(T,B),     literal_tuple(B,L), hold(L).

Overwriting meta-simple-ex2.lp


The next cell should return the unique answer set `{a}`.

In [27]:
! clingo example3.lp --output=reify | clingo - meta-simple-ex2.lp 0 -V0


a b
SATISFIABLE


### Exercise 3
In the next cell, write a modification of `meta-simple.lp` such that:
* if a rule has some literals in its body, then the body is interpreted as a disjunction of literals, and *not* as a conjunction of literals.

Consider the program `example4.lp`:

In [28]:
%%file example4.lp
{a;b}.
c :- a,b.

Overwriting example4.lp


Under this interpretation, this program becomes:
```
{a;b}.
c :- a.
c :- b.
```
and leads to four answer sets: `{}`, `{a,c}`, `{b,c}`, and `{a,b,c}`.

Note that the modification should only apply to bodies that contain some literal.

In [29]:
%%file meta-simple-ex3.lp

conjunction(B) :- literal_tuple(B),
        hold(L) : literal_tuple(B, L), L > 0;
    not hold(L) : literal_tuple(B,-L), L > 0.

  hold(A) : atom_tuple(H,A)   :- rule(disjunction(H),normal(B)), conjunction(B).
{ hold(A) : atom_tuple(H,A) } :- rule(     choice(H),normal(B)), conjunction(B).

#show.
#show T : output(T,B), not literal_tuple(B,_).
#show T : output(T,B),     literal_tuple(B,L), hold(L).

Overwriting meta-simple-ex3.lp


The next cell should return the answer sets `{}`, `{a,c}`, `{b,c}`, and `{a,b,c}`.

In [30]:
! clingo example4.lp --output=reify | clingo - meta-simple-ex3.lp 0 -V0

a b c

b
a
SATISFIABLE


## An issue

Let us go back to Exercise 2, and consider the program `example5.lp`:

In [31]:
%%file example5.lp
a.
b :- not a.

Overwriting example5.lp


In Exercise 2, this should be interpreted as:
```
{a}.
{b} :- a.
```
and lead to three answer sets: `{}`, `{a}`, and `{a,b}`.

Let's see what do we obtain with the meta-encoding of Exercise 2:

In [ ]:
! clingo example5.lp --output=reify | clingo - meta-simple-ex2.lp 0 -V0

As you should see, using the meta-encoding of Exercise 2 we still obtain the unique answer set `{a}`: how is this possible? 

The warnings printed by clingo give us a clue on this.
They tell us that the atoms of `literal_tuple(B,L)` do not occur in any head.

Let us have a look at the ground instantiation and the reification of `example5.lp`:

In [33]:
! clingo example5.lp --mode=gringo

asp 1 0 0
1 0 1 1 0 0
4 1 a 0
0


In [34]:
! python aspif/aspif-pretty-printer.py example5.lp

1.
#show a.


In [35]:
! clingo example5.lp --output=reify

atom_tuple(0).
atom_tuple(0,1).
literal_tuple(0).
rule(disjunction(0),normal(0)).
output(a,0).


Observe that there is a fact `literal_tuple(0)` but no facts `literal_tuple(0,_)`. 
In this way, `0` refers to the empty body.

What happened here? 
* The grounder component of clingo first prints the fact `a.` as `1.`
* Then it handles the rule `b :- not a`.
* But instead of generating `2 :- -1`, it knows that `a` is a fact, and therefore 
the body of the rule will never be applicable, so it simply skips the rule.
* Then, when printing the `#show` directives, the grounder knows that `a` is a fact and prints it directly as 
`#show a.`

Observe how, in the reified program, there is no trace of the original rule `b :- not a`.
Hence, there is nothing we can do in our meta-encoding to interpret this rule differently.

There are some ways to overcome this issue, using `#external` directives, for example.
But we will not dwelve on them here.

We just have to remember that
* **the meta-encoding is applied to the ground instantiation, not to the original clingo program with variables**.

## Weight rules

Consider the program `example6.lp`:

In [36]:
%%file example6.lp
{a;b;c;d}.
e :- a, 30 #sum{ 20:b; 30:c; 40:d }.
:- not e.

Writing example6.lp


You can reify it using the following command, but instead of doing that, please jump over the cell and have a look at the commented version below.

In [ ]:
! clingo example6.lp --output=reify

In [ ]:
%%file example6-reified.lp

% { 1; 2; 3; 4}.
atom_tuple(0).
atom_tuple(0,1).
atom_tuple(0,2).
atom_tuple(0,3).
atom_tuple(0,4).
literal_tuple(0).
rule(choice(0),normal(0)).

% 5 :- 30 { 20:2; 30:3; 40:4 }.
atom_tuple(1).
atom_tuple(1,5).
weighted_literal_tuple(0).
weighted_literal_tuple(0,2,20).
weighted_literal_tuple(0,3,30).
weighted_literal_tuple(0,4,40).
rule(disjunction(1),sum(0,30)).

% 6 :- 1, 5.
atom_tuple(2).
atom_tuple(2,6).
literal_tuple(1).
literal_tuple(1,5).
literal_tuple(1,1).
rule(disjunction(2),normal(1)).

% :- not 6.
atom_tuple(3).
literal_tuple(2).
literal_tuple(2,-6).
rule(disjunction(3),normal(2)).

% #show a : 1.
literal_tuple(3).
literal_tuple(3,1).
output(a,3).

% #show b : 2.
literal_tuple(4).
literal_tuple(4,2).
output(b,4).

% #show c : 3.
literal_tuple(5).
literal_tuple(5,3).
output(c,5).

% #show d : 4.
literal_tuple(6).
literal_tuple(6,4).
output(d,6).

% #show e : 6.
literal_tuple(7).
literal_tuple(7,6).
output(e,7).

Observe how the rule 
```
e :- a, 30 #sum{ 20:b; 30:c; 40:d }.
```
is separated in two parts.

One part defines the aggregate using an auxiliary atom numbered `5`:
```
5 :- 30 { 20:2; 30:3; 40:4 }.
```
and another handles the rule using that auxiliary atom:
```
6 :- 1, 5.
```


### Exercise 4

In the next cell, modify the meta-encoding to fix the weights of weight rules to the value of `10`.

Under this interpretation, program `example6.lp` becomes:
```
{a;b;c;d}.
e :- a, 30 #sum{ 10:b; 10:c; 10:d }.
:- not e.
```
and has a unique answer set `{a,b,c,d,e}`.

In [37]:
%%file meta-ex4.lp 

conjunction(B) :- literal_tuple(B),
        hold(L) : literal_tuple(B, L), L > 0;
    not hold(L) : literal_tuple(B,-L), L > 0.

body(normal(B)) :- rule(_,normal(B)), conjunction(B).
body(sum(B,G))  :- rule(_,sum(B,G)),
    #sum { W,L :     hold(L), weighted_literal_tuple(B, L,W), L > 0 ;
           W,L : not hold(L), weighted_literal_tuple(B,-L,W), L > 0 } >= G.

  hold(A) : atom_tuple(H,A)   :- rule(disjunction(H),B), body(B).
{ hold(A) : atom_tuple(H,A) } :- rule(     choice(H),B), body(B).

#show.
#show T : output(T,B), conjunction(B).

Writing meta-ex4.lp


The following command should lead to the unique answer set `{a,b,c,d,e`}.

In [38]:
! clingo example6.lp --output=reify | clingo - meta-ex4.lp 0 -V0

a d e
a d b e
a c e
a c b e
a d c e
a d c b e
SATISFIABLE


To finish the exercise, modify again the meta-encoding in such a way that the lower bound for weight rules is always `100`. One this is done, the previous cell should return no answer set.

## Relaxing integrity constraints

A logic program may have zero, one or many answer sets. 
If the program has zero answer sets, we may want to recover part of its *partial* answer sets. 

This is specially useful whenever we are debugging an incorrect program 
that has no answer sets, since in this case the `UNSATISFIABLE` result provided by clingo
tells us nothing about why that program is unsatisfiable.

One way of recovering *partial* answer sets of an unsatisfiable logic program 
is to relax the integrity constraints of the program.
Instead of requiring that no integrity constraint is violated, 
we require that the least number of integrity constraints is violated.

For example, consider the program `example7.lp`, that has no answer sets:

In [39]:
%%file example7.lp
{ a(1); a(2) }.
:- 1 { a(X) }.
:-   { a(X) } 1.
#show a/1.

Overwriting example7.lp


In [40]:
! clingo example7.lp -V0

UNSATISFIABLE


We would like to intepret the program as this one:

In [41]:
%%file example7-relaxed.lp
{ a(1); a(2) }.
violated(ic1) :- 1 { a(X) }.
violated(ic2) :-   { a(X) } 1.
:~ violated(R). [1,R]
#show a/1.
#show violated/1.

Overwriting example7-relaxed.lp


We can enumerate all optimal solutions with option `0 --opt-mode=optN`.

In [42]:
! clingo example7-relaxed.lp 0 --opt-mode=optN -V0

violated(ic2)
Optimization: 1
violated(ic2)
Optimization: 1
a(1) a(2) violated(ic1)
Optimization: 1
OPTIMUM FOUND


This gives us two optimal solutions of cost 1: `{violated(ic2)}` and `{a(1), a(2), violated(ic1)}`.

The first answer is printed twice: 
first when it is found, 
and second when all optimal solutions are enumerated.

The `violated(R)` atoms indicate which integrity constraints where violated.

As can be seen, each optimal solution violated exactly one integrity constraint.

### Exercise 5.

In the next cell, modify `meta.lp` to relax the integrity constraints, as we did in the previous example:

In [43]:
%%file meta-relaxed.lp

conjunction(B) :- literal_tuple(B),
        hold(L) : literal_tuple(B, L), L > 0;
    not hold(L) : literal_tuple(B,-L), L > 0.

body(normal(B)) :- rule(_,normal(B)), conjunction(B).
body(sum(B,G))  :- rule(_,sum(B,G)),
    #sum { W,L :     hold(L), weighted_literal_tuple(B, L,W), L > 0 ;
           W,L : not hold(L), weighted_literal_tuple(B,-L,W), L > 0 } >= G.

  hold(A) : atom_tuple(H,A)   :- rule(disjunction(H),B), body(B).
{ hold(A) : atom_tuple(H,A) } :- rule(     choice(H),B), body(B).

#show.
#show T : output(T,B), conjunction(B).

Writing meta-relaxed.lp


The following command should return two optimal solutions, like above with `example7-relaxed.lp`:

In [ ]:
! clingo example7.lp --output=reify | clingo - meta-relaxed.lp 0 --opt-mode=optN